# Local frequency analyses

In [ ]:
# Basic imports
import hvplot.xarray  # noqa
import numpy as np
import xarray as xr
import xdatasets

import xhydro as xh
import xhydro.frequency_analysis as xhfa

## Data extraction and preparation

In this example, we will perform a frequency analysis using historical timeseries data collected from various sites. Our first step is to acquire a hydrological dataset. For this, we utilize the [xdatasets](https://hydrologie.github.io/xdatasets/notebooks/getting_started.html) library to retrieve hydrological data from the [Ministère de l'Environnement, de la Lutte contre les changements climatiques, de la Faune et des Parcs](https://www.cehq.gouv.qc.ca/atlas-hydroclimatique/stations-hydrometriques/index.htm) in Québec, Canada. Our query will focus on stations with IDs beginning with `020`, and specifically those with natural flow pattern. Some stations have information on water levels, but will will only process streamflow data.

Alternatively, users can provide their own `xarray.DataArray`. When preparing the data for frequency analysis, it must adhere to the following requirements:

- The dataset must include a `time` dimension.
- If there is a 1D spatial dimension (e.g., `id` in the example below), it must contain an attribute `cf_role` with the value `timeseries_id`.
- The variable must have at least a `units` attribute. Although additional attributes, such as `long_name` and `cell_methods`, are not strictly required, they are expected by `xclim`, which is invoked during the frequency analysis. Missing these attributes will trigger warnings.


In [ ]:
ds = (
    xdatasets.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["020*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds = ds.rename({"streamflow": "q"})
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["q"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

# Clean some of the coordinates that are not needed for this example
ds = ds.drop_vars([c for c in ds.coords if c not in ["id", "time", "name"]])

# For the purpose of this example, we keep only 2 stations
ds = ds.isel(id=slice(3, 5))
ds

In [ ]:
ds.q.dropna("time", how="all").hvplot(
    x="time", grid=True, widget_location="bottom", groupby="id"
)

## Acquiring block maxima

The `xhydro.indicators.get_yearly_op` function can be used to extract block maxima from a time series. This function provides several options for customizing the extraction process, such as selecting the desired time periods and more. In the following section, we will present the main arguments available to help tailor the block maxima extraction to your needs.

In [ ]:
help(xh.indicators.get_yearly_op)

### a) Defining seasons

Seasons can be defined using indexers compatible with `xclim.core.calendar.select_time`. Four types of indexers are currently supported:

- `month`: A sequence of month numbers (e.g., `[1, 2, 12]` for January, February, and December).
- `season`: A sequence of season abbreviations, with options being `'DJF'`, `'MAM'`, `'JJA'`, and `'SON'` (representing Winter, Spring, Summer, and Fall, respectively).
- `doy_bounds`: A sequence specifying the inclusive bounds of the period in terms of day of year (e.g., `[152, 243]`).
- `date_bounds`: Similar to `doy_bounds`, but using a month-day format (`'%m-%d'`), such as `["01-15", "02-23"]`.

When using `xhydro.indicators.get_yearly_op` to calculate block maxima, the indexers should be grouped within a dictionary and passed to the `timeargs` argument. The dictionary keys should represent the requested period (e.g., `'winter'`, `'summer'`) and will be appended to the variable name. Each dictionary entry can include the following:

- The indexer, as defined above (e.g., `"date_bounds": ["02-11", "06-19"]`).
- (Optional) An annual resampling frequency. This is mainly used for indexers that span across the year. For example, setting `"freq": "YS-DEC"` will start the year in December instead of January.

In [ ]:
# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {
        "season": ["DJF"],
        "freq": "YS-DEC",
    },  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {},
}

### b) Defining criteria for missing values

The `xhydro.indicators.get_yearly_op` function also includes two arguments, `missing` and `missing_options`, which allow you to define tolerances for missing data. These arguments leverage `xclim` to handle missing values, and the available options are detailed in the [xclim documentation](https://xclim.readthedocs.io/en/stable/checks.html#missing-values-identification).


In [ ]:
import xclim

print(xclim.core.missing.__doc__)

### c) Simple example

Let's start with a straightforward example using the `timeargs` dictionary defined earlier. In this case, we will set a tolerance where a maximum of 15% of missing data in a year will be considered acceptable for the year to be regarded as valid.


In [ ]:
ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

ds_4fa

In [ ]:
ds_4fa.q_max_summer.dropna("time", how="all").hvplot(
    x="time", grid=True, widget_location="bottom", groupby="id"
)

### d) Advanced example: Using custom seasons per year or per station

Customizing date ranges for each year or each station is not directly supported by `xhydro.indicators.get_yearly_op`. However, users can work around this limitation by masking their data before calling the function. When applying this approach, be sure to adjust the `missing` argument to accommodate the changes in the data availability.

In this example, we will define a season that starts on a random date in April and ends on a random date in June. Since we will mask almost the entire year, the tolerance for missing data should be adjusted accordingly. Instead of setting a general tolerance for missing data, we will use the `at_least_n` method to specify that at least 45 days of data must be available for the period to be considered valid.


In [ ]:
nyears = np.unique(ds.time.dt.year).size
dom_start = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)
dom_end = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)

mask = xr.zeros_like(ds["q"])
for y in dom_start.year.values:
    # Random mask of dates per year, between April and June.
    mask.loc[
        {
            "time": slice(
                str(y) + "-04-" + str(dom_start.sel(year=y).item()),
                str(y) + "-06-" + str(dom_end.sel(year=y).item()),
            )
        }
    ] = 1

In [ ]:
mask.hvplot(x="time", grid=True, widget_location="bottom", groupby="id")

In [ ]:
# The name of the indexer will be used to identify the variable created here
timeargs_custom = {"custom": {}}

# We use .where() to mask the data that we want to ignore
masked = ds.where(mask == 1)
# Since we masked almost all the year, our tolerance for missing data should be changed accordingly
missing = "at_least_n"
missing_options = {"n": 45}

# We use xr.merge() to combine the results with the previous dataset.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            masked,
            op="max",
            timeargs=timeargs_custom,
            missing=missing,
            missing_options=missing_options,
        ),
    ]
)

In [ ]:
ds_4fa.q_max_custom.dropna("time", how="all").hvplot(
    x="time", grid=True, widget_location="bottom", groupby="id"
)

### e) Alternative variable: Computing volumes

Frequency analysis can also be applied to volumes, following a similar workflow to that of streamflow data. The main difference is that if we're starting with streamflow data, we must first convert it into volumes using `xhydro.indicators.compute_volume` (e.g., going from `m3 s-1` to `m3`). Additionally, if necessary, the `get_yearly_op` function includes an argument, `interpolate_na`, which can be used to interpolate missing data before calculating the sum.


In [ ]:
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["q"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            ds,
            op="sum",
            input_var="volume",
            timeargs=timeargs_vol,
            missing="pct",
            missing_options={"tolerance": 0.15},
            interpolate_na=True,
        ),
    ]
)

In [ ]:
ds_4fa.volume_sum_spring.dropna("time", how="all").hvplot(
    x="time", grid=True, widget_location="bottom", groupby="id"
)

## Local frequency analysis

After extracting the raw data, such as annual maximums or minimums, the local frequency analysis is performed in three steps:

1. Use `xhydro.frequency_analysis.local.fit` to determine the best set of parameters for a given number of statistical distributions.
2. (Optional) Use `xhydro.frequency_analysis.local.criteria` to compute goodness-of-fit criteria and assess how well each statistical distribution fits the data.
3. Use `xhydro.frequency_analysis.local.parametric_quantiles` to calculate return levels based on the fitted parameters.

To speed up the Notebook, we'll only perform the analysis on a subset of variables.

In [ ]:
help(xhfa.local.fit)

The `fit` function enables the fitting of multiple statistical distributions simultaneously, such as `["genextreme", "pearson3", "gumbel_r", "expon"]`. Since different distributions have varying parameter sets (and sometimes different naming conventions), `xHydro` handles this complexity by using a `dparams` dimension, filling in NaN values where needed. When the results interact with `SciPy`, such as the `parametric_quantiles` function, only the relevant parameters for each distribution are passed. The selected distributions are stored in a newly created `scipy_dist` dimension.


In [ ]:
params = xhfa.local.fit(ds_4fa[["q_max_spring", "volume_sum_spring"]], min_years=15)

params

Criteria like AIC (Akaike Information Criterion), BIC (Bayesian Information Criterion), and AICC (Corrected AIC) are valuable tools for comparing the fit of different statistical models. These criteria balance the goodness-of-fit of a model with its complexity, helping to avoid overfitting. AIC and AICC are particularly useful when comparing models with different numbers of parameters, while BIC tends to penalize complexity more heavily, making it more conservative. Lower values of these criteria indicate better model performance, with AICC being especially helpful in small sample sizes. By using these metrics, we can objectively determine the most appropriate model for our data.

These three criteria can be accessed using `xhydro.frequency_analysis.local.criteria`. The results are added to a new `criterion` dimension. In this example, the AIC, BIC, and AICC all provide a weak indication that the Generalized Extreme Value (GEV) distribution is the best fit for the data, though the Gumbel distribution may also be a suitable choice. Conversely, the Pearson III failed to converge and the exponential distribution was rejected based on these criteria, suggesting that they do not adequately fit the data.


In [ ]:
help(xhfa.local.criteria)

In [ ]:
criteria = xhfa.local.criteria(ds_4fa[["q_max_spring", "volume_sum_spring"]], params)

criteria["q_max_spring"].isel(id=0).compute()

Finally, return periods can be obtained using `xhfa.local.parametric_quantiles`.

In [ ]:
help(xhfa.local.parametric_quantiles)

In [ ]:
rp = xhfa.local.parametric_quantiles(params, t=[2, 20, 100])

rp.load()

In a future release, plotting will be managed by a dedicated function. For now, we demonstrate the process using preliminary utilities in this notebook.

The function `xhfa.local._prepare_plots` generates the data points required to visualize the results of the frequency analysis. If `log=True`, it will return log-spaced x-values between `xmin` and `xmax`. Meanwhile, `xhfa.local._get_plotting_positions` calculates plotting positions for all variables in the dataset. It accepts `alpha` and `beta` arguments. For typical values, refer to the [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mstats.plotting_positions.html). By default, `(0.4, 0.4)` is used, which corresponds to the quantile unbiased method (Cunnane).


In [ ]:
data = xhfa.local._prepare_plots(params, xmin=1, xmax=1000, npoints=50, log=True)
pp = xhfa.local._get_plotting_positions(ds_4fa[["q_max_spring"]])

In [ ]:
# Let's plot the observations
p1 = data.q_max_spring.hvplot(
    x="return_period", by="scipy_dist", grid=True, groupby=["id"], logx=True
)
data.q_max_spring.hvplot(
    x="return_period", by="scipy_dist", grid=True, groupby=["id"], logx=True
)

# Let's now plot the distributions
p2 = pp.hvplot.scatter(
    x="q_max_spring_pp",
    y="q_max_spring",
    grid=True,
    groupby=["id"],
    logx=True,
)
pp.hvplot.scatter(
    x="q_max_spring_pp",
    y="q_max_spring",
    grid=True,
    groupby=["id"],
    logx=True,
)

# And now combining the plots
p1 * p2

## Uncertainties

Uncertainties are an important aspect of frequency analysis and should be considered when interpreting results. These uncertainties often stem from data quality, the choice of distribution, and the estimation of parameters. While visualizations can provide insights into the model fit, it’s crucial to quantify and account for uncertainties, such as confidence intervals for parameter estimates, to ensure robust conclusions.

In order to manage computational intensity, we will focus on a single catchment and limit the analysis to the two distributions that appeared to best fit the data.

In [ ]:
ds_4fa = ds_4fa.sel(id="020602")[["q_max_spring"]]
params = params.sel(id="020602", scipy_dist=["genextreme", "gumbel_r"])[
    ["q_max_spring"]
]

### a) Bootstrapping the observations

One method for quantifying uncertainties is to bootstrap the observations. In this example, we will perform bootstrapping a small number of times to illustrate the process, though in practice, a higher number of iterations (e.g., 5000) is recommended to obtain more reliable estimates. Bootstrapping resamples the observed data with replacement to generate multiple datasets, which can then be used to assess the variability in the model's parameters and results.

This can be accomplished by calling `xhydro.frequency_analysis.uncertainties.bootstrap_obs`.


In [ ]:
help(xhfa.uncertainties.bootstrap_obs)

In [ ]:
ds_4fa_boot_obs = xhfa.uncertainties.bootstrap_obs(ds_4fa, n_samples=35)
ds_4fa_boot_obs

This process will add a new dimension, `samples`, to the dataset. When used in conjunction with `xhydro.frequency_analysis.local.fit`, a new set of parameters will be computed for each sample. As a result, bootstrapping can become computationally expensive, especially as the number of bootstrap iterations increases.


In [ ]:
params_boot_obs = xhfa.local.fit(
    ds_4fa_boot_obs, distributions=["genextreme", "gumbel_r"]
)
rp_boot_obs = xhfa.local.parametric_quantiles(params_boot_obs, t=[2, 20, 100])
rp_boot_obs.load()

### b) Bootstrapping the distributions

In this approach, rather than resampling the observations directly, we resample the fitted distributions to estimate the uncertainty. This method allows us to assess the variability in the fitted distributions' parameters. As with the previous example, we will perform a minimal number of bootstrap iterations to reduce computational load, but in practice, a higher number of iterations would provide more robust estimates of uncertainty.

This can be accomplished by calling `xhydro.frequency_analysis.uncertainties.bootstrap_dist`. Unlike `bootstrap_obs`, this method does not support lazy evaluation and requires a specific function for the fitting step: `xhydro.frequency_analysis.uncertainties.fit_boot_dist`.

In [ ]:
help(xhfa.uncertainties.bootstrap_dist)

In [ ]:
help(xhfa.uncertainties.fit_boot_dist)

In [ ]:
tmp_values = xhfa.uncertainties.bootstrap_dist(ds_4fa, params, n_samples=35)
params_boot_dist = xhfa.uncertainties.fit_boot_dist(tmp_values)
params_boot_dist

In [ ]:
rp_boot_dist = xhfa.local.parametric_quantiles(
    params_boot_dist.load(), t=[2, 20, 100]
)  # Lazy computing is not supported
rp_boot_dist

### c) Comparison

Let's show the difference between both approaches.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
fig.set_figheight(4)
fig.set_figwidth(15)

# Subset the data
rp_orig = rp.q_max_spring.sel(id="020602", scipy_dist="genextreme")
boot_obs = rp_boot_obs.q_max_spring.sel(scipy_dist="genextreme")
boot_dist = rp_boot_dist.q_max_spring.sel(scipy_dist="genextreme")

# Original fit
ax.plot(
    rp_orig.return_period.values,
    rp_orig,
    "black",
    label="Original fit",
)

ax.plot(
    boot_obs.return_period.values,
    boot_obs.quantile(0.5, "samples"),
    "red",
    label="Bootstrapped observations",
)
boot_obs_05 = boot_obs.quantile(0.05, "samples")
boot_obs_95 = boot_obs.quantile(0.95, "samples")
ax.fill_between(
    boot_obs.return_period.values, boot_obs_05, boot_obs_95, alpha=0.2, color="red"
)

ax.plot(
    boot_dist.return_period.values,
    boot_dist.quantile(0.5, "samples"),
    "blue",
    label="Bootstrapped distribution",
)
boot_dist_05 = boot_dist.quantile(0.05, "samples")
boot_dist_95 = boot_dist.quantile(0.95, "samples")
ax.fill_between(
    boot_dist.return_period.values, boot_dist_05, boot_dist_95, alpha=0.2, color="blue"
)

plt.xscale("log")
plt.grid(visible=True)
plt.xlabel("Return period (years)")
plt.ylabel("Streamflow (m$^3$/s)")
ax.legend()